### PROBLEM 4: Knowledge Base Question Answering

Given is knowledge graph with entities and relations, questions with starting entity and answers, and their word embedding . For each question, navigate the graph from the start entity outwards until you find appropriate answer entities.

- there might be multiple answers correct, use F1 to evaluate
- utils functions (similarity, load_graphs) are given, but you can choose not to use them
- answers are given to be used for evaluation only
- your strategy should be a graph traversal augmented with scoring of paths; you might discard paths not promising along the way. This is similar to a focused crawl strategy.
- for simplicity, the questions are picked so that the answer is always at the end of the relevant path (not intermediary) 

In [10]:
# Importing the required libraries

import heapq
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

import gensim
import numpy as np
import pandas as pd
import networkx as nx
from nltk import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Setting seed for reproducability

seed = 42
np.random.seed(seed)

In [3]:
# Data URLs

BASE_DIR = Path().cwd().parent
DATA_DIR = BASE_DIR / "data" / "kbqa"

GRAPH_URL = DATA_DIR / "graph.txt"
ANNOTATIONS_URL = DATA_DIR / "annotations.txt"
WORD_EMBEDDINGS_URL = str(DATA_DIR / "word2vec_train_dev.dat")

In [4]:
# Function to get the cosine similarity between a relation and query
# Note: Be sure to prepend the relation with ns:
word2vec_model = gensim.models.Word2Vec.load(WORD_EMBEDDINGS_URL)
def get_rel_score_word2vecbase(rel, query):
    if rel not in word2vec_model.wv:
        return 0.0
    words = word_tokenize(query.lower())
    w_embs = []
    for w in words:
        if w in word2vec_model.wv:
            w_embs.append(word2vec_model.wv[w])
    return np.mean(cosine_similarity(w_embs, [word2vec_model.wv[rel]]))


# Function to load the graph from file
def load_graph():
    # Preparing the graph
    graph = defaultdict(list)
    with open(GRAPH_URL) as graph_file:
        lines = graph_file.readlines()
        for line in lines:
            line = eval(line[:-1])
            graph[line[0]].append([line[1], line[2]])
    return graph


# Function to load the queries from file
# Preparing the queries
def load_queries():
    queries = []
    with open(ANNOTATIONS_URL) as annot_file:
        lines = annot_file.readlines()
        for line in lines:
            line = eval(line[:-1])
            queries.append(line)
    return queries

In [5]:
knowledge_graph = load_graph()
knowledge_graph

defaultdict(list,
            {'m.01_d4': [['location.citytown.postal_codes', 'm.07nqmhw'],
              ['location.citytown.postal_codes', 'm.07nqmhf'],
              ['location.citytown.postal_codes', 'm.01_6pxw'],
              ['location.location.contains', 'm.021czc'],
              ['location.citytown.postal_codes', 'm.07nqmj9'],
              ['travel.travel_destination.tourist_attractions', 'm.02vv_6w'],
              ['location.citytown.postal_codes', 'm.01_71cx'],
              ['location.citytown.postal_codes', 'm.07nqmfx'],
              ['location.citytown.postal_codes', 'm.01_6y3n'],
              ['location.citytown.postal_codes', 'm.07nqmj2'],
              ['location.citytown.postal_codes', 'm.01_6p6k'],
              ['location.citytown.postal_codes', 'm.0gs227r'],
              ['location.citytown.postal_codes', 'm.01_6s_6'],
              ['travel.travel_destination.tourist_attractions', 'm.02rvwrv'],
              ['location.location.contains', 'm.01ky66'],
      

In [6]:
queries = load_queries()
queries

[[1,
  'what time zones are there in the us',
  'm.09c7w0',
  [['ns:m.09c7w0', 'ns:location.location.time_zones', '?x']],
  'United States of America',
  [{'AnswerType': 'Entity',
    'AnswerArgument': 'm.027wj2_',
    'EntityName': 'Samoa Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.027wjl3',
    'EntityName': 'Chamorro Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.02fqwt',
    'EntityName': 'Central Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.02hcv8',
    'EntityName': 'Eastern Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.02hczc',
    'EntityName': 'Mountain Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.02lcqs',
    'EntityName': 'Pacific Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.02lcrv',
    'EntityName': 'Alaska Time Zone'},
   {'AnswerType': 'Entity',
    'AnswerArgument': 'm.02lctm',
    'EntityName': 'Hawaii-Aleutian Time Zone'},
   {'AnswerType': 'Enti

In [7]:
question = queries[3]
idx, eng_question, start_node, question_query, eng_start_node, actual_answer = question

question

[4,
 'what war was george washington associated with',
 'm.034rd',
 [['ns:m.034rd', 'ns:military.military_commander.military_commands', '?y'],
  ['?y', 'ns:military.military_command.military_conflict', '?x']],
 'George Washington',
 [{'AnswerType': 'Entity',
   'AnswerArgument': 'm.014pd3',
   'EntityName': 'Battle of Brandywine'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.014s1b',
   'EntityName': 'Siege of Yorktown'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.01bh42',
   'EntityName': 'Treaty of Alliance'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.01g09p',
   'EntityName': 'Battle of Long Island'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.01g0dx',
   'EntityName': 'Battle of Princeton'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.01gphc',
   'EntityName': 'Braddock Expedition'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.01hgpt',
   'EntityName': 'Battle of Trenton'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.01jnys',
   

In [8]:
def traverse_knowlegde_graph(knowledge_graph: defaultdict, start_node: str):
    SIMILARITY_THRESHOLD = 0.3

    pq = []
    pq.append((0, start_node))
    
    visited = set()
    answer_nodes = set()
    
    while len(pq) > 0:
        sim_score, curr_node = heapq.heappop(pq)
    
        if curr_node in visited:
            continue
    
        visited.add(curr_node)
    
        sim_score = -1 * sim_score
        if sim_score >= SIMILARITY_THRESHOLD:
            answer_nodes.add(curr_node)
    
        for neighbors in knowledge_graph[curr_node]:
            relation, adj_entity = neighbors
            
            relation = "ns:" + relation
            sim_score = get_rel_score_word2vecbase(relation, eng_question)
            
            heapq.heappush(pq, (-sim_score, adj_entity))

    return answer_nodes

In [11]:
TP, FP, FN = 0, 0, 0
for query in tqdm(queries, desc="Answering Queries"):
    idx, eng_question, start_node, question_query, eng_start_node, actual_answers = query
    
    answer_nodes = traverse_knowlegde_graph(knowledge_graph, start_node)
    actual_answers = set(x["AnswerArgument"] for x in actual_answers)

    TP += len(answer_nodes.intersection(actual_answers))
    FP += len(answer_nodes.difference(actual_answers))
    FN += len(actual_answers.difference(answer_nodes))

f1_score = (2 * TP) / (2 * TP + FP + FN)
print(f"F1 score of the custom Knowledge Base Question Answering: {f1_score:.3f}")

Answering Queries: 100%|████████████████████████| 56/56 [00:29<00:00,  1.88it/s]

F1 score of the custom Knowledge Base Question Answering: 0.336
